# Pre Load Models

In [2]:
import sys
sys.path.append('../')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from keras.layers import Dense, Dropout

import tensorflow as tf
import numpy as np
from tqdm import tqdm
from keras.layers import Dense, Dropout, GRU

import pandas as pd
import tensorflow as tf
import numpy as np

from machine_learning.models import DoubleProduction, DoubleConcatProduction, DoubleExtraProduction, DoubleExtraConcatProduction
from data_processing.batch_generator import load_test_set
tf.random.set_seed(42)

### Dataset

In [3]:
path = 'data'
BATCH_SIZE = 1024
SEQUENCE_LENGTH = 100

In [4]:
sample_transaction = np.load(f'data/test/transactions.npy')[0]

# TRAIN
train_labels = np.load(f'{path}/train/all_transaction_labels.npy').astype(float)
train_set = tf.data.Dataset.from_tensor_slices(
    (np.load(f'{path}/train/transactions.npy'), 
     train_labels)
).batch(1)

# TEST
test_labels = np.load(f'{path}/test/all_transaction_labels.npy').astype(float)
test_set = tf.data.Dataset.from_tensor_slices(
    (np.load(f'{path}/test/transactions.npy'), 
     test_labels)
).batch(1)

# Double

In [5]:
model = DoubleProduction()

In [6]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=[
            tf.keras.metrics.BinaryAccuracy(),
            tf.keras.metrics.TruePositives(), 
            tf.keras.metrics.TrueNegatives(),
            tf.keras.metrics.FalsePositives(), 
            tf.keras.metrics.FalseNegatives(),
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall()])

# initialize weights
model(np.expand_dims(sample_transaction, axis=0))
model.load_weights(f'machine_learning/saved_models/{model.name}_{14}.keras')


In [7]:
model.reset_gru()

In [8]:
model.evaluate(train_set)

1296675/1296675 [==============================] - 683s 526us/step - loss: 0.0205 - binary_accuracy: 0.9953 - true_positives: 2555.0000 - true_negatives: 1288073.0000 - false_positives: 1096.0000 - false_negatives: 4951.0000 - precision: 0.6998 - recall: 0.3404


[0.020471647381782532,
 0.9953365325927734,
 2555.0,
 1288073.0,
 1096.0,
 4951.0,
 0.6998082995414734,
 0.34039434790611267]

In [9]:
tf.print(model.category_gru.shared_states, summarize=-1)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0.968557715 1 -0.907712281 0.478679448 -1 -0.165565312 -1 1 1 1 -0.996242702 1 0.987215698 1 -1 1 1 1 1 -0.999427736 0.192443788 1 1 -1 0.999997258 -1 1 1 -1 -1 1 -0.152285174 -0.00525893318 -0.999997556 1 0.999902964 0.841560781 0.999983668 -0.0848994255 1 -1 -0.945939481 0.986543536 1 0.999999702 0.999998391 -1]
 [1 0.873869777 1 -0.477733314 1 -1 -0.0410506 -1 1 1 1 -0.438200533 1 0.999999464 1 -0.999148369 1 1 1 1 -1 0.00992718432 1 1 -1 0.999704182 -1 1 -0.999998927 -1 -1 0.999986112 -0.823151171 -0.00138527446 -0.999352694 1 0.999740958 0.999505222 0.999999702 0.999996424 1 -1 0.999998629 -0.962480903 1 0.999914527 1 -1]
 [1 0.94645226 1 -0.602550626 0.999980867 -1 -5.48361186e-06 -1 1 1 1 0.307358086 1 0.997247279 1 0.981125 1 1 1 1 0.344316274 6.78879223e-05 1 -0.991442442 -1 0.493140906 -1 1 1 -1 -1 0.993941367 0 -0.00144753081 -0.149378449 1 0.993480623 0.928517044 1 -0.9999

In [13]:
weight_path = f'machine_learning/pre_loaded_models/{model.name}.keras'
model.save_weights(
    filepath=weight_path,
    save_format='h5'
)

In [14]:
model.evaluate(test_set)

555719/555719 [==============================] - 299s 538us/step - loss: 0.0164 - binary_accuracy: 0.9965 - true_positives: 670.0000 - true_negatives: 553095.0000 - false_positives: 479.0000 - false_negatives: 1475.0000 - precision: 0.5831 - recall: 0.3124


[0.01641053520143032,
 0.9964838624000549,
 670.0,
 553095.0,
 479.0,
 1475.0,
 0.5831157565116882,
 0.31235432624816895]

# Remaining Models

In [5]:
model_list = [(DoubleConcatProduction, 19), (DoubleExtraProduction, 19), (DoubleExtraConcatProduction, 12)]
for model_class, epoch in model_list:
    model = model_class()

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=[
            tf.keras.metrics.BinaryAccuracy(),
            tf.keras.metrics.TruePositives(), 
            tf.keras.metrics.TrueNegatives(),
            tf.keras.metrics.FalsePositives(), 
            tf.keras.metrics.FalseNegatives(),
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall()])

    # initialize weights
    model(np.expand_dims(sample_transaction, axis=0))
    model.load_weights(f'machine_learning/saved_models/{model.name}_{epoch}.keras')

    model.reset_gru()
    model.evaluate(train_set)

    weight_path = f'machine_learning/pre_loaded_models/{model.name}.keras'
    model.save_weights(
        filepath=weight_path,
        save_format='h5'
    )

1296675/1296675 [==============================] - 712s 549us/step - loss: 0.0220 - binary_accuracy: 0.9942 - true_positives_2: 2504.0000 - true_negatives_2: 1286704.0000 - false_positives_2: 2465.0000 - false_negatives_2: 5002.0000 - precision_2: 0.5039 - recall_2: 0.3336
